In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pathlib
import cdmetadl
from tqdm import tqdm
import numpy as np
import cdmetadl.helpers.general_helpers
import cdmetadl.dataset
import cdmetadl.samplers
import cdmetadl.confidence
import cdmetadl.notebooks.helpers
path_to_dataset = "/fastdata/vilab24/meta-album"


PROJECT_DIR = pathlib.Path(cdmetadl.__file__).parent.parent


dataset_path = pathlib.Path(path_to_dataset)
dataset_info_dict = cdmetadl.helpers.general_helpers.check_datasets(dataset_path, ["AWA"])

dataset = cdmetadl.dataset.ImageDataset("-", dataset_info_dict["AWA"])


n_way_sampler = cdmetadl.samplers.ValueSampler(value=5)
k_shot_sampler = cdmetadl.samplers.ValueSampler(value=4)

task = dataset.generate_task(n_way_sampler, k_shot_sampler, query_size=4)

## Monte Carlo Dropout Confidence Estimation

## Finetuning

In [3]:
confidence_estimator = cdmetadl.confidence.MCDropoutConfidenceEstimator(num_samples=1000)
samples_conf_scores = 10
model_module = cdmetadl.helpers.general_helpers.load_module_from_path(PROJECT_DIR / "baselines/finetuning/model.py")

for T in (1, 10, 50, 100, 250, 500, 750, 1000, 1500, 2000):
    learner = model_module.MyLearner()
    learner.load("/fastdata/vilab25/output/final_a/training/dropout_k_5/finetuning/cross-domain/model")
    learner.T = T
    conf_scores = np.array([confidence_estimator.estimate(learner, task.support_set)[1] for _ in tqdm(range(samples_conf_scores))])

    correlation_between_conf_scores = np.corrcoef(conf_scores)

    avg_correlation = (sum(np.corrcoef(conf_scores).flatten()) - samples_conf_scores)/(samples_conf_scores**2 - samples_conf_scores)
    print(f"T: {T}, Average Correlation between runs: {round(avg_correlation, 4)}")



100%|██████████| 10/10 [00:25<00:00,  2.56s/it]


T: 1, Average Correlation between runs: -0.0842


100%|██████████| 10/10 [00:24<00:00,  2.41s/it]


T: 10, Average Correlation between runs: 0.0406


100%|██████████| 10/10 [00:27<00:00,  2.72s/it]


...

T: 1000, Average Correlation between runs: 0.9935


100%|██████████| 10/10 [01:26<00:00,  8.70s/it]


T: 1500, Average Correlation between runs: 0.9952


100%|██████████| 10/10 [01:45<00:00, 10.57s/it]

T: 2000, Average Correlation between runs: 0.9987


## MAML

In [17]:
confidence_estimator = cdmetadl.confidence.MCDropoutConfidenceEstimator(num_samples=1000)
samples_conf_scores = 10


for T in (1, 5, 10, 20, 30, 50, 70, 100, 150, 200):
    model_module = cdmetadl.helpers.general_helpers.load_module_from_path(PROJECT_DIR / "baselines/maml/model.py")
    learner = model_module.MyLearner()
    learner.load("/fastdata/vilab25/output/final_a/training/dropout_k_5/maml/cross-domain/model")
    learner.T = T
    conf_scores = np.array([confidence_estimator.estimate(learner, task.support_set)[1] for _ in tqdm(range(samples_conf_scores))])

    correlation_between_conf_scores = np.corrcoef(conf_scores)

    avg_correlation = (sum(np.corrcoef(conf_scores).flatten()) - samples_conf_scores)/(samples_conf_scores**2 - samples_conf_scores)
    print(f"T: {T}, Average Correlation between runs: {round(avg_correlation, 4)}")




AttributeError: 'ResNet' object has no attribute 'forward_weights'